## **Onet Scrapping**

In [ ]:
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb

wget -N https://storage.googleapis.com/chrome-for-testing-public/132.0.6821.2/mac-arm64/chromedriver-mac-arm64.zip -P /tmp/
unzip -o /tmp/chromedriver-mac-arm64 -d /tmp/
chmod +x /tmp/chromedriver-mac-arm64/chromedriver
mv /tmp/chromedriver-linux64/chromedriver /usr/local/bin/chromedriver
pip install selenium chromedriver_autoinstaller

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,200 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,599 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-sec

In [ ]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

from concurrent.futures import ThreadPoolExecutor, as_completed
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import chromedriver_autoinstaller
import pandas as pd
import time

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # this is must
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chromedriver_autoinstaller.install()

#driver = webdriver.Chrome(options=chrome_options)

'/usr/local/lib/python3.10/dist-packages/chromedriver_autoinstaller/131/chromedriver'

In [ ]:
def init_driver():
    driver = webdriver.Chrome(options=chrome_options)
    return driver

In [ ]:
def fetch_description_and_skills(job):
    driver = init_driver()
    try:
        # Open the job link
        driver.get(job["Link"])

        # Fetch job description
        description_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div#content > p"))
        )
        job["Description"] = description_element.text

        # Fetch technical skills
        skills_section = driver.find_elements(By.CSS_SELECTOR, "div.order-2.flex-grow-1 > b")
        # Filter out empty strings and clean text
        job["Skills"] = [skill.text.strip() for skill in skills_section if skill.text.strip()]
    except Exception as e:
        job["Description"] = f"Description not found due to error: {e}"
        job["Skills"] = [f"Error: {e}"]
    finally:
        driver.quit()

In [ ]:
def scrape_jobs():
    driver = init_driver()
    driver.get("https://www.onetonline.org/find/all")
    driver.implicitly_wait(10)

    job_data = []
    table_rows = driver.find_elements(By.CSS_SELECTOR, "tbody tr")
    for row in table_rows:
        job_zone = row.find_element(By.CSS_SELECTOR, "td:nth-child(1)").text
        code = row.find_element(By.CSS_SELECTOR, "td:nth-child(2)").text
        occupation = row.find_element(By.CSS_SELECTOR, "td:nth-child(3)").text
        occupation_element = row.find_element(By.CSS_SELECTOR, "td:nth-child(3) a")
        occupation_link = occupation_element.get_attribute("href")
        job_data.append({"Job Zone": job_zone, "Code": code, "Occupation": occupation, "Link": occupation_link, "Skills": []})

    driver.quit()  # Close the initial driver
    return job_data

In [ ]:
def main():
    # Step 1: Scrape job list
    job_data = scrape_jobs()

    # Step 2: Fetch job descriptions and skills in parallel
    with ThreadPoolExecutor(max_workers=5) as executor:
        futures = [executor.submit(fetch_description_and_skills, job) for job in job_data]
        for future in as_completed(futures):
            future.result()  # Ensure all tasks are completed

    # Step 3: Collect all unique technical skills
    all_skills = set()
    for job in job_data:
        all_skills.update(job["Skills"])

    # Step 4: Create skill mapping and save outputs
    skill_mapping = {skill: idx for idx, skill in enumerate(sorted(all_skills))}
    job_to_skills = [{"Job Code": job["Code"], "Skills": [skill_mapping[skill] for skill in job["Skills"] if skill in skill_mapping]} for job in job_data]

    # All technical skills with unique IDs
    skills_df = pd.DataFrame(list(skill_mapping.items()), columns=["Skill", "Skill ID"])
    #skills_df.to_csv("technical_skills.csv", index=False)

    # Job to skill mapping
    job_skills_df = pd.DataFrame(job_to_skills)
    #job_skills_df.to_csv("job_to_skills.csv", index=False)

    # Job data with descriptions
    job_df = pd.DataFrame(job_data)
    #job_df.to_csv("jobdata.csv", index=False)

    #print("Scraping complete! Files saved: 'technical_skills.csv', 'job_to_skills.csv', 'job_data.csv'")

if __name__ == "__main__":
    main()

KeyboardInterrupt: 